<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# FEC Dashboard
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas%20Dashboard/Naas_Dashboard_Financial_Consolidation.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a><br><br><a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=&template=template-request.md&title=Tool+-+Action+of+the+notebook+">Template request</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Naas+Dashboard+-+Financial+Consolidation:+Error+short+description">Bug report</a> | <a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Start_data_product.ipynb" target="_parent">Generate Data Product</a>

**Tags:** #naasdashboard #plotly #dash #naas #asset #automation #analytics #snippet #datavizualisation

**Authors:** [Alexandre STEVENS](https://www.linkedin.com/in/alexandrestevenspbix/) | [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/) | [Jeremy Ravenel](https://www.linkedin.com/in/jeremyravenel/)

**Description:** This notebook provides a comprehensive dashboard for financial consolidation and analysis.

## Input

### Import libraries

In [ ]:
try:
    import dash
except:
    !pip install dash --user
    import dash
from dash import Dash, html, dcc, Input, Output, State, dash_table
try:
    import dash_bootstrap_components as dbc
except:
    !pip install dash_bootstrap_components --user
    import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px
import os
import pandas as pd
from naas_drivers import gsheet
from dash_bootstrap_components._components.Container import Container
from plotly.subplots import make_subplots
import random
from naas_data_product import *
import base64
import datetime
import io
import re
import time
from naas_drivers import naasauth

### Setup Variables

In [ ]:
# Inputs
APP_TITLE = "FEC Dashboard"
APP_LOGO = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRszy3bMYjE7-_YpiDDqWrLEGR-wJKEbLyff-VQMJOZoA&s"

sidebar_intro_text = "Bienvenue dans la section d'import des Fichiers des Écritures Comptables (FEC). Vous pouvez importer vos fichiers FEC ici, les prévisualiser et ensuite exécuter le pipeline de traitement de données pour intégrer ces données dans votre tableau de bord."
sidebar_step1_text = "Cliquez sur ce bouton pour sélectionner et télécharger votre fichier FEC depuis votre système local vers votre serveur Naas privé."
sidebar_step2_text = "Une fois que vous avez validé  les imports, cliquez sur le bouton ci-dessous pour lancer le pipeline de traitement des données FEC.Veuillez noter que le processus peut prendre un certain temps, en fonction de la taille du fichier et de la complexité des données. Vous recevrez une notification une fois le processus terminé."
sidebar_support_text = "Si vous rencontrez des problèmes lors du téléchargement, de la prévisualisation ou du traitement des fichiers FEC, veuillez contacter notre équipe d'assistance à l’adresse: "
sidebar_text_style = {
    "text-align": "justify",
}


CARD_COL_STYLE = {
    "className": "g-2",
    "xs": 12,
    "sm": 12,
    "md": 12,
    "lg": 2,
    "xl": 2
}
CHART_COL_STYLE = {
    "xs": 12,
    "sm": 12,
    "md": 12,
    "lg": 6,
    "xl": 6,
}
BACKGROUND_COLOR = "#f8f9fa"
COLOR_RED = "#ff3938" #"#ea5545"
COLOR_PINK = "#f46a9b"
COLOR_ORANGE = "#ef9b20"
COLOR_LIGHT_ORANGE = "#edbf33"
COLOR_YELLOW = "#ede15b"
COLOR_LIGHT_GREEN = "#bdcf32"
COLOR_GREEN = "#87bc45"
COLOR_BLUE = "#27aeef"
COLOR_PURPLEQ = "#b33dc6"

output_dir = "/home/ftp/FEC-engine/outputs"
path_ref_entite =  f"{output_dir}/FEC/referentiel_entite"
path_ref_periode =  f"{output_dir}/FEC/referentiel_periode"
path_kpis =  f"{output_dir}/FEC/dataset_kpis"
path_evolution_ca =  f"{output_dir}/FEC/dataset_evolution_ca"
path_tresorerie =  f"{output_dir}/FEC/dataset_tresorerie"
path_charges =  f"{output_dir}/FEC/dataset_charges"
path_bilan =  f"{output_dir}/FEC/dataset_bilan"
entity_demo =  "000000000"
period_demo = "201812"

naas_account = naasauth.connect().user.me().get("username")

# Outputs
DASH_PORT = 8050 # Defining the port of the dashboard

## Model

### Data & Charts

#### KPIS

In [ ]:
df_kpis = get_last_df(path_kpis)
print("Nb row:", len(df_kpis))
df_kpis.head(1)

#### Data : Evolution CA

In [ ]:
df_evolution_ca = get_last_df(path_evolution_ca)
print("Nb row:", len(df_evolution_ca))
df_evolution_ca.head(1)

#### Chart 1 : Evolution CA

In [ ]:
def create_linechart(
    df_init,
    label_col="LABEL",
    value_col="VALUE",
    value_col_d="VALUE_D",
    xaxis_title=None,
    yaxis_title=None,
):
    # Init
    fig = go.Figure()
    if len(df_init) == 0:
        return fig
    df = df_init.copy()
    df[value_col] = df[value_col] / 1000 #K€
    
    # Data
    df_n = df[df["GROUP"] == "N"].reset_index(drop=True)
    df_n_1 = df[df["GROUP"] == "N-1"].reset_index(drop=True)

    # Add traces
    fig.add_trace(
        go.Scatter(
            name="N",
            x=df_n[label_col],
            y=df_n[value_col],
            marker=dict(size=8, symbol='circle', color=COLOR_BLUE),
            hoverinfo="text",
            text=df_n[value_col_d],
            line=dict(width=2.5),
        )
    )
    fig.add_trace(
        go.Scatter(
            name="N-1",
            x=df_n_1[label_col],
            y=df_n_1[value_col],
            marker=dict(size=8, symbol='circle', color=COLOR_ORANGE),
            hoverinfo="text",
            text=df_n_1[value_col_d],
            line=dict(width=2.5),
        )
    )
    # Add figure title
    fig.update_layout(
        title_font=dict(family="Arial", size=18, color="black"),
        legend=
        dict(
            x=0.5, #Modifier la valeur x pour ajuster l'alignement horizontal de la légende (0.0 à gauche, 1.0 à droite)
            y=1, ## Modifier la valeur y pour ajuster l'alignement vertical de la légende (0.0 en bas, 1.0 en haut)
            orientation="h",
            xanchor='center',  # Alignement horizontal de la légende ('left', 'center', 'right')
            yanchor='bottom'  # Alignement vertical de la légende ('top', 'middle', 'bottom')
        ),
        plot_bgcolor="#ffffff",
        paper_bgcolor="white",
        xaxis_title=xaxis_title,
        xaxis_title_font=dict(family="Arial", size=12, color="black"),
        xaxis={"type": "category"},
        margin=dict(
            l=50,  # Marge gauche
            r=50,  # Marge droite
            t=50,  # Marge supérieure
            b=50,  # Marge inférieure
        ),
    )

    # Set y-axes titles
    fig.update_yaxes(
        title_text=yaxis_title,
        title_font=dict(family="Arial", size=12, color="black"),
    )
    return fig

fig_linechart = create_linechart(
    df_evolution_ca[(df_evolution_ca["ENTITY"] == entity_demo) & (df_evolution_ca["PERIOD"] == period_demo)],
    label_col="LABEL",
    value_col="VALUE_CUM",
    value_col_d="VALUE_CUM_D"
)
fig_linechart

#### Data: Trésorerie

In [ ]:
df_tresorerie = get_last_df(path_tresorerie)
print("Nb row:", len(df_tresorerie))
df_tresorerie.head(1)

#### Chart 2 : Position de trésorerie

In [ ]:
def create_barlinechart(
    df,
    label='LABEL',
    value_line="VALUE_LINE",
    value_line_d="VALUE_LINE_D",
    value_cash_in="CASH_IN",
    value_cash_in_d="CASH_IN_D",
    value_cash_out="CASH_OUT",
    value_cash_out_d="CASH_OUT_D",
    xaxis_title=None,
    yaxis_title_r=None,
    yaxis_title_l=None,
):
    # Create figure with secondary y-axis
#     fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig = go.Figure()
    if len(df) == 0:
        return fig

    # Add traces
    fig.add_trace(
        go.Bar(
            name="Encaissement",
            x=df[label],
            y=df[value_cash_in],
            hovertext=df[value_cash_in_d],
            hoverinfo="text",
            marker=dict(color=COLOR_GREEN),
        ),
#         secondary_y=False,
    )
    fig.add_trace(
        go.Bar(
            name="Décaissement",
            x=df[label],
            y=df[value_cash_out],
            hovertext=df[value_cash_out_d],
            hoverinfo="text",
            marker=dict(color=COLOR_RED),
        ),
#         secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(
            name="Position",
            x=df[label],
            y=df[value_line],
            hovertext=df[value_line_d],
            hoverinfo="text",
            mode="lines+markers",
            marker=dict(size=8, symbol='circle', color=COLOR_BLUE),
            line=dict(width=2.5),
        ),
#         secondary_y=True,
    )

    # Update layout
    fig.update_layout(
        title_font=dict(family="Arial", size=18, color="black"),
        legend=
        dict(
            x=0.5, #Modifier la valeur x pour ajuster l'alignement horizontal de la légende (0.0 à gauche, 1.0 à droite)
            y=1, ## Modifier la valeur y pour ajuster l'alignement vertical de la légende (0.0 en bas, 1.0 en haut)
            orientation="h",
            xanchor='center',  # Alignement horizontal de la légende ('left', 'center', 'right')
            yanchor='bottom'  # Alignement vertical de la légende ('top', 'middle', 'bottom')
        ),
        plot_bgcolor="#ffffff",
        paper_bgcolor="white",
        xaxis_title=xaxis_title,
        xaxis_title_font=dict(family="Arial", size=12, color="black"),
        xaxis={"type": "category"},
        margin=dict(
            l=50,  # Marge gauche
            r=50,  # Marge droite
            t=50,  # Marge supérieure
            b=50,  # Marge inférieure
    ),
    )

    # Set y-axes titles
    fig.update_yaxes(
        title_text=yaxis_title_r,
        title_font=dict(family="Arial", size=12, color="black"),
#         secondary_y=False,
    )
#     fig.update_yaxes(
#         title_text=yaxis_title_l,
#         title_font=dict(family="Arial", size=12, color="black"),
#         secondary_y=True,
#     )
#     fig.update_traces(showlegend=True)
    return fig


fig_barline_chart = create_barlinechart(
    df_tresorerie[(df_tresorerie["ENTITY"] == entity_demo) & (df_tresorerie["PERIOD"] == period_demo)],
    label="MONTH_INDEX"
)
fig_barline_chart

#### Data: Charges

In [ ]:
df_charges = get_last_df(path_charges)
print("Nb row:", len(df_charges))
df_charges.tail(1)

#### Chart 3 : Répartition des charges

In [ ]:
def create_hbarhcart(
    df_init,
    label="LABEL",
    value="VALUE",
    value_d="VALUE_D",
    value_comp="VALUE_N-1",
    value_comp_d="VALUE_N-1_D",
    xaxis_title=None,
    yaxis_title_r=None,
    yaxis_title_l=None,
):
    # Init
    fig = go.Figure()
    df = df_init.copy()
    if len(df_init) == 0:
        return fig
    
    # Data prep
    to_group = ["ENTITY", "PERIOD", label]
    to_agg = {value: "sum", value_comp: "sum"}
    df = df.groupby(to_group, as_index=False).agg(to_agg)
    df[value] = round(df[value], 0)
    df = df.sort_values(by=value, ascending=True)
    df[label] = df[label].str.replace("_", " ").str.lower().str.capitalize()
    df[value_d] = (df[value] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")
    df[value_comp_d] = (df[value_comp] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")
    
    # Add trace N-1
    fig.add_trace(
        go.Bar(
            name="N-1",
            x=df[value_comp],
            y=df[label],
            orientation="h",
            text=df[value_comp_d],
            textposition="inside",
            marker=dict(color=COLOR_ORANGE)
        )
    )
    
    # Add trace N
    fig.add_trace(
        go.Bar(
            name="N",
            x=df[value],
            y=df[label],
            orientation="h",
            text=df[value_d],
            textposition="inside",
            marker=dict(color=COLOR_BLUE)
        )
    )
    
    # Update layout
    fig.update_layout(
    title_font=dict(family="Arial", size=18, color="black"),
    plot_bgcolor="#ffffff",
    xaxis_title=None,
    xaxis_showticklabels=False,
    yaxis_title=None,
    margin_pad=10,
    bargap=0.1,  # gap between bars of adjacent location coordinates.
    bargroupgap=0.2,# gap between bars of the same location coordinate.
    margin=dict(
        l=50,  # Marge gauche
        r=50,  # Marge droite
        t=50,  # Marge supérieure
        b=50,  # Marge inférieure
    ),
    legend=
        dict(
            x=0.5, #Modifier la valeur x pour ajuster l'alignement horizontal de la légende (0.0 à gauche, 1.0 à droite)
            y=1, ## Modifier la valeur y pour ajuster l'alignement vertical de la légende (0.0 en bas, 1.0 en haut)
            orientation="h",
            xanchor='center',  # Alignement horizontal de la légende ('left', 'center', 'right')
            yanchor='bottom'  # Alignement vertical de la légende ('top', 'middle', 'bottom')
        ),
    
    )
    return fig

fig_hbarhcart_chart = create_hbarhcart(
    df_charges[(df_charges["ENTITY"] == entity_demo) & (df_charges["PERIOD"] == period_demo)],
    label="RUBRIQUE_N1"
)
fig_hbarhcart_chart

#### Data: Bilan

In [ ]:
df_bilan = get_last_df(path_bilan)
print("Nb row:", len(df_bilan))
df_bilan.head(1)

#### Chart 4 : Répartition Bilan

In [ ]:
def create_treemapchart(
    df,
    entity="ENTITY",
    scenario="PERIOD",
    value="VALUE",
    xaxis_title=None,
    yaxis_title_r=None,
    yaxis_title_l=None,
):
    # Init
    fig = go.Figure()
    if len(df) == 0:
        return fig
    
    # Data
    def prep_data(df_init):
        # Init
        df = df_init.copy()
        df_output = pd.DataFrame()
        levels = ["RUBRIQUE_N0", "RUBRIQUE_N1", "RUBRIQUE_N2"]
        to_group = [entity, scenario]
        to_agg = {value: "sum"}
        parents = []
        
        # Loop
        for index, level in enumerate(levels):
            # Create map dict
            map_parent = {}
            for i, row in df.iterrows():
                if index == 0:
                    row_parent = ""
                else:
                    row_parent = row[levels[index-1]]
                map_parent[row[level]] = row_parent
            # Init
            tmp_df = df.copy()
            group_level = to_group + [level]
            tmp_df = tmp_df.groupby(group_level, as_index=False).agg(to_agg).rename(columns={level: "LABEL"})
            tmp_df.insert(loc=3, column="PARENT", value=tmp_df["LABEL"].map(map_parent))
            df_output = pd.concat([df_output, tmp_df])
            
        df_output["LABEL"] = df_output["LABEL"].astype(str).str.replace("_", " ").str.lower().str.capitalize()
        df_output["PARENT"] = df_output["PARENT"].astype(str).str.replace("_", " ").str.lower().str.capitalize()
        return df_output.reset_index(drop=True)
    
    # Init Chart
    fig = make_subplots(
        cols=2,
        rows=1,
        horizontal_spacing=0.02,
        specs=[[{"type": "treemap", "rowspan": 1}, {"type": "treemap"}]],
    )
    
    # Create "Asset" treemap
    df_asset = prep_data(df[df["RUBRIQUE_N0"] == "ACTIF"].reset_index(drop=True))
    fig.add_trace(
        go.Treemap(
            labels=df_asset["LABEL"],
            parents=df_asset["PARENT"],
            values=df_asset["VALUE"],
            textinfo="label+value+percent parent",
            marker_colorscale="Blues",
            hoverinfo="label+value+percent parent",
            branchvalues="total",
        ),
        row=1,
        col=1,
    )
    
    # Create "Liability" treemap
    df_liability = prep_data(df[df["RUBRIQUE_N0"] == "PASSIF"].reset_index(drop=True))
    fig.add_trace(
        go.Treemap(
            labels=df_liability["LABEL"],
            parents=df_liability["PARENT"],
            values=df_liability["VALUE"],
            textinfo="label+value+percent parent",
            marker_colorscale="oranges",
            hoverinfo="label+value+percent parent",
            branchvalues="total",
        ),
        row=1,
        col=2,
    )
    return fig

fig_treemap_chart = create_treemapchart(
    df_bilan[(df_bilan["ENTITY"] == entity_demo) & (df_bilan["PERIOD"] == period_demo)],
)
fig_treemap_chart

### Design App

#### Dropdown data: Ref Entities

In [ ]:
df_entities = get_last_df(path_ref_entite)
entities = sorted(df_entities["ENTITY"].unique().tolist())

# Entity's dropdown list
dropdown_entity = dcc.Dropdown(
    id="entity",
    options=[{"label": i, "value": i} for i in entities],
    placeholder="Entity",
    value=entities[0],
)

#### Dropdown data: Ref Scenarios

In [ ]:
df_scenarios = df_entities[df_entities["ENTITY"] == entities[0]]
scenarios = [{"label": r["PERIOD_D"], "value": r["PERIOD"]} for i, r in df_scenarios.iterrows()]
scenario_init = df_scenarios.loc[0, "PERIOD"]

# Scenario's dropdown list
dropdown_scenario = dcc.Dropdown(
    id="scenario",
    options=scenarios,
    placeholder="Scenario",
    value=scenario_init
)

#### Create Card

In [ ]:
# Function to create card
def create_card_col(
    title,
    value,
    value_id,
    var,
    var_id
):
    # Create card
    card = dbc.Col(
        dbc.Card(
            dbc.CardBody(
                [
                    html.P(
                        title,
                        className="card-title",
                        style={
                            "font-size": "15px",
                            "padding-bottom": "5px",
                            "text-align": "center",
                            "font-weight": "bold",
                            "color": "#181A1C",
                        },
                    ),
                    html.P(
                        children=value,
                        className="card-text",
                        style={
                            "font-size": "26px",
                            "text-align": "center",
                            "font-weight": "bold",
                            "color": "#181A1C"
                        },
                        id=value_id,
                    ),
                    
                    html.P(
                        children=var,
                        className="card-text",
                        style={
                            "font-size": "13px",
                            "text-align": "center",
                            "padding-top": "0px",
                            "color": "#181A1C",
                        },
                        id=var_id,
                    ),
                    
                ]
            ),
            color="#fbfbfb",
            inverse=True,
        ),
            
            style={
                "padding-left": "10px", 
                "padding-right": "10px"
            },
        **CARD_COL_STYLE,
    )
    return card

#### Create Chart Card

In [ ]:
def create_chart(
    title,
    subtitle,
    chart_id,
    fig,
    analysis_id="",
    analysis="",
):  
    # Create card
    card = dbc.Col(
        dbc.Card(
            dbc.CardBody(
                [
                    html.H5(
                        title,
                        style={
                            "padding-top": "20px",
                            "margin-left": "20px",
                            "font-weight": "bold",
                            "font-size": "15px",
                        },
                    ),
                    html.H5(
                        subtitle,
                        style={
                            "padding-top": "4px",
                            "margin-left": "20.5px",
                            "font-size": "12px",
                        },
                    ),
                    dcc.Graph(
                        figure=fig,
                        id=chart_id,
                        config={
                            "displayModeBar": False,
                        }
                    ),
                    html.H5(
                        "Interprétation IA:",
                        style={
                            "font-size": "12px",
                        },
                    ),
                    html.P(
                        analysis,
                        id=analysis_id,
                        style={
                            "text-align": "justify",
                            "font-size": "11px",
                        },
                    ),
                ],
            ),
            color="#fbfbfb",
            inverse=False,
        ),
        style={
            "display": "inline",
            "padding-left": "10px", 
            "padding-right": "10px"
        },
        **CHART_COL_STYLE
    )
    return card

#### Create App Layout

In [ ]:
app = dash.Dash(
    requests_pathname_prefix=f'/user/{os.environ.get("JUPYTERHUB_USER")}/proxy/{DASH_PORT}/',
    external_stylesheets=[dbc.themes.BOOTSTRAP],
    meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1.0"}
    ],
)

# app = dash.Dash() if you are not in Naas
app.title = APP_TITLE
app._favicon = "naas.ico"
app.layout = html.Div(
    [
        # represents the browser address bar
        dcc.Location(id='url', refresh=False),
        
        # Navbar
        dbc.Navbar(
            dbc.Container(
                [
                    html.A(
                        # Use row and col to control vertical alignment of logo / brand
                        dbc.Row(
                            [
                                dbc.Col(html.Img(src=APP_LOGO, height="50")),
                                dbc.Col(dbc.NavbarBrand(APP_TITLE, className="ms-2")),
                            ],
                            align="center",
                            className="g-0",
                        ),
                    ),
                    dbc.NavbarToggler(id="navbar-toggler", n_clicks=0),
                    dbc.Collapse(
                        dbc.Nav(
                            [
                                html.Div(
                                    [
                                        html.Div(className="w-100"),
                                        html.Div(className="w-100"),
                                        html.Div(className="w-100"),
                                        html.Div(dropdown_entity, className="w-100", id="dropdown_entity"),
                                        html.Div(dropdown_scenario, className="w-100", id="dropdown_scenario"),
                                        html.Div(
                                        [
                                            dbc.Button("⚙️", id="open-offcanvas", n_clicks=0, color="white"),
                                            dbc.Offcanvas(
                                                [
                                                    html.P(
                                                        sidebar_intro_text,
                                                        style=sidebar_text_style
                                                    ),
                                                    html.H5(
                                                        "Etape 1"
                                                    ),
                                                    html.P(
                                                        sidebar_step1_text,
                                                        style=sidebar_text_style
                                                    ),
                                                    html.Br(),
                                                    # Add button to upload data
                                                    dcc.Upload(
                                                        id="upload-data",
                                                        children=dbc.Button(
                                                            [
                                                                html.A("Télécharger FEC"),
                                                            ],
                                                            id='upload-button',
                                                            color="light",
                                                            className="d-grid gap-2 col-6 mx-auto",
                                                        ),
                                                        # Allow multiple files to be uploaded
                                                        multiple=True,
                                                    ),
                                                    html.Br(),
                                                    html.Div(id="output-data-upload"),
                                                    html.Br(),
                                                    
                                                    html.H5(
                                                        "Etape 2"
                                                    ),
                                                    html.P(
                                                        [
                                                            sidebar_step2_text
                                                        ],
                                                        style=sidebar_text_style
                                                    ),

                                                    # SPACE
                                                    html.Br(),

                                                    # Button to update pipeline
                                                    html.Div(
                                                        [
                                                            html.A(
                                                                [
                                                                    dbc.Button(
                                                                        "Exécuter Pipeline FEC",
                                                                        id="loading-button",
                                                                        n_clicks=0,
                                                                        className="d-grid gap-2 col-6 mx-auto",
                                                                        color="light",
                                                                    ),
                                                                    dbc.Spinner(html.Div(id="loading-output")), 
                                                                ],
                                                                id="refresh",
                                                            )

                                                        ]
                                                    ),
                                                    html.Br(),
                                                    html.H5(
                                                        "Support"
                                                    ),
                                                    html.P(
                                                        [
                                                            sidebar_support_text,
                                                            html.A("fec@naas.ai", href="mailto:fec@naas.ai")
                                                        ],
                                                        style=sidebar_text_style
                                                    ),
                                                ],
                                                id="offcanvas",
                                                title="Import des FEC",
                                                is_open=False,
                                                placement="end",
                                            ),
                                        ]
                                    )
                                    ],
                                    className="pt-1 pb-1 d-grid gap-2 d-md-flex w-100",
                                )
                            ],
                            className="ms-auto w-100",
                            navbar=True,
                        ),
                        id="navbar-collapse",
                        navbar=True,
                        is_open=False,
                    ),
                ],
                # Display
                style={
                    "display": "contents"
                }
            ),
            color="#181A1C",
            dark=True,
        ),
        
        # HKPIS
        dbc.Row(
            [
                create_card_col("CHIFFRE D'AFFAIRES", "180", "value1", "15", "var1"),
                create_card_col("MARGE BRUTE", "180", "value2", "15", "var2"),
                create_card_col("EBE", "180", "value3", "15", "var3"),
                create_card_col("BFR", "180", "value4", "15", "var4"),
                create_card_col("CREANCES CLIENTS", "180", "value5", "15", "var5"),
                create_card_col("DETTES FOURNISSEURS", "180", "value6", "15", "var6"),
            ],
            style={
                "padding-top": "20px",
            },
            className="g-2 d-flex align-items-center",
        ),
        
        # SPACE
        html.Br(),
        
        # Chart: Row1
        dbc.Row(
            [
                create_chart(
                    "EVOLUTION DU CHIFFRE D'AFFAIRES",
                    "Comparaison N vs N-1, mensuel (en k€)",
                    "fig_1",
                    fig_linechart,
                    "ai_1",
                    
                ),
                create_chart(
                    "POSITION DE TRESORERIE",
                    "Encaissement, décaissement et position de trésorerie, mensuel (en k€)",
                    "fig_2",
                    fig_barline_chart,
                    "ai_2",
                ),
            ],
            className="g-2 d-flex align-items-center",
        ),
        
        # SPACE
        html.Br(),
        
        # Chart: Row2
        dbc.Row(
            [
                create_chart(
                    "CHARGES",
                    "Comparaison N vs N-1 (en k€)",
                    "fig_3",
                    fig_hbarhcart_chart,
                    "ai_3",
                ),
                create_chart(
                    "BILAN",
                    "Structure bilantielle (en k€)",
                    "fig_4",
                    fig_treemap_chart,
                    "ai_4",
                ),
            ],
            className="g-2 d-flex align-items-center",
        ),
        
        # SPACE
        html.Br(),
    ],
    style={"backgroundColor": BACKGROUND_COLOR}
)


@app.callback(
    Output("offcanvas", "is_open"),
    Input("open-offcanvas", "n_clicks"),
    [State("offcanvas", "is_open")],
)
def toggle_offcanvas(n1, is_open):
    if n1:
        return not is_open
    return is_open

def parse_contents(contents, filename, date):
    df = pd.DataFrame()
    content_type, content_string = contents.split(",")
    decoded = base64.b64decode(content_string)
    try:
        if re.match("^\d{9}FEC\d{8}.txt", filename):
            # Assume that the user uploaded a CSV file
            df = pd.read_csv(
                io.StringIO(decoded.decode("ISO-8859-1")),
                sep="\t",
                decimal=",",
                header=0
            )
            
            # Save dataframe
            df.to_csv(
                f"/home/ftp/FEC-engine/inputs/{filename}",
                encoding="ISO-8859-1",
                sep="\t",
                decimal=",",
                index=False
            )
            return html.Div(
                [
                    f"✅ Fichier FEC chargé dans Naas:\n '{filename}' ({len(df)} lignes). ",
                ]

            ), df
    except Exception as e:
        print(e)
    return html.Div([f"❌ Fichier non conforme au format FEC attendu: '{filename}'"]), df

@app.callback(
    [
        Output("output-data-upload", "children"),
        Output("upload-button", "style"),
    ],
    Input("upload-data", "contents"),
    State("upload-data", "filename"),
    State("upload-data", "last_modified"),
)
def update_output(list_of_contents, list_of_names, list_of_dates):
    childrens = []
    style={"padding": "10px", "background-color": COLOR_BLUE, "color": "white"}
    df = pd.DataFrame()
    if list_of_contents is not None:
        for c, n, d in zip(list_of_contents, list_of_names, list_of_dates):
            children, tmp_df = parse_contents(c, n, d)
            childrens.append(children)
            df = pd.concat([df, tmp_df])
        if len(df) == 0:
            color_button = COLOR_RED
        else:
            color_button = "#47dd88"
        style={"padding": "10px", "background-color": color_button, "color": "white"}
    return childrens, style

@app.callback(
    [
        Output("loading-output", "children"),
        Output("loading-button", "children"),
        Output("loading-button", "href"),
        Output("loading-button", "style"),
        Output("dropdown_entity", "children"),
    ],
    [
        Input("loading-button", "n_clicks"),
    ]
)
def load_output(n):
    loading_output = ""
    button_name = "Exécuter Pipeline FEC"
    href = None
    style = {"padding": "10px", "background-color": COLOR_BLUE, "color": "white"}
    if n:
        # Run pipeline
        pipeline_path = f"{MODELS_PATH}/__pipeline__.ipynb"
        %run $pipeline_path
        time.sleep(2)
        button_name = "Raffraichir la page"
        href = f"https://app.naas.ai/user/{naas_account}/proxy/{DASH_PORT}/"
        style = {"padding": "10px", "background-color": "#47dd88", "color": "white"}
        
    df_entities = get_last_df(path_ref_entite)
    entities = sorted(df_entities["ENTITY"].unique().tolist())
    
    # Entity's dropdown list
    dropdown_entity = dcc.Dropdown(
        id="entity",
        options=[{"label": i, "value": i} for i in entities],
        placeholder="Entity",
        value=entities[0],
    )
    return None, button_name, href, style, [dropdown_entity]


@app.callback(
    [
        Output("dropdown_scenario", "children"),
    ],
    [
        Input("entity", "value"),
    ]
)
def load_scenario(value):
    df_entities = get_last_df(path_ref_entite)
    df_scenarios = df_entities[df_entities["ENTITY"].astype(str) == str(value)].reset_index(drop=True)
    scenarios = [{"label": r["PERIOD_D"], "value": r["PERIOD"]} for i, r in df_scenarios.iterrows()]
    scenario_init = df_scenarios.loc[0, "PERIOD"]
    # Scenario's dropdown list
    dropdown_scenario = dcc.Dropdown(
        id="scenario",
        options=scenarios,
        placeholder="Scenario",
        value=scenario_init
    )
    return [dropdown_scenario]


# add callback for toggling the collapse on small screens
@app.callback(
    Output("navbar-collapse", "is_open"),
    [Input("navbar-toggler", "n_clicks")],
    [State("navbar-collapse", "is_open")],
)
def toggle_navbar_collapse(n, is_open):
    if n:
        return not is_open
    return is_open


# add callback to filter data in cards and charts
@app.callback(
    [
        Output("value1", "children"),
        Output("value2", "children"),
        Output("value3", "children"),
        Output("value4", "children"),
        Output("value5", "children"),
        Output("value6", "children"),
        Output("var1", "children"),
        Output("var2", "children"),
        Output("var3", "children"),
        Output("var4", "children"),
        Output("var5", "children"),
        Output("var6", "children"),
        Output("var1", "style"),
        Output("var2", "style"),
        Output("var3", "style"),
        Output("var4", "style"),
        Output("var5", "style"),
        Output("var6", "style"),
        Output("fig_1", "figure"),
        Output("fig_2", "figure"),
        Output("fig_3", "figure"),
        Output("fig_4", "figure"),
        Output("ai_1", "children"),
        Output("ai_2", "children"),
        Output("ai_3", "children"),
        Output("ai_4", "children"),
    ],
    [
        Input("entity", "value"),
        Input("scenario", "value")
    ],
)

def multi_outputs(entity, scenario):
    # Init
    fig1 = None
    fig2 = None
    fig3 = None
    fig4= None 
    ai1 = ""
    ai2 = ""
    ai3 = ""
    ai4 = ""

    # Get AI analysis
    def get_ai_analysis(df=pd.DataFrame()):
        ai = ""
        if len(df) > 0:
            if "AI_ANALYSIS" in df.columns:
                ai = df.loc[0, "AI_ANALYSIS"]
        return ai
    
    # Get entity and scenario
    print(entity, scenario)
    if entity is None and scenario is None:
        raise PreventUpdate
        
    # Create EVOLUTION CA fig and get AI analysis
    df_ca = get_last_df(path_evolution_ca)
    df_ca = df_ca[
        (df_ca["ENTITY"].astype(str) == str(entity)) & (df_ca["PERIOD"].astype(str) == str(scenario))
    ].reset_index(drop=True)
    fig1 = create_linechart(
        df_ca,
        label_col="LABEL",
        value_col="VALUE_CUM",
        value_col_d="VALUE_CUM_D"
    )
    ai1 = get_ai_analysis(df_ca)
    
    # Get Trésorerie dataframe
    df_treso = get_last_df(path_tresorerie)
    df_treso = df_treso[
        (df_treso["ENTITY"].astype(str) == str(entity)) & (df_treso["PERIOD"].astype(str) == str(scenario))
    ].reset_index(drop=True)
    fig2 = create_barlinechart(
        df_treso,
        label="MONTH_INDEX"
    )

    # Get Charges dataframe
    dfcharges = get_last_df(path_charges)
    dfcharges = dfcharges[
        (dfcharges["ENTITY"].astype(str) == str(entity)) & (dfcharges["PERIOD"].astype(str) == str(scenario))
    ].reset_index(drop=True)
    fig3 = create_hbarhcart(
        dfcharges,
        label="RUBRIQUE_N1"
    )
    
    # Get Bilan dataframe
    dfbilan = get_last_df(path_bilan)
    dfbilan = dfbilan[
        (dfbilan["ENTITY"].astype(str) == str(entity)) & (dfbilan["PERIOD"].astype(str) == str(scenario))
    ].reset_index(drop=True)
    fig4 = create_treemapchart(
        dfbilan,
    )

    # Get HKPIs dataframe
    dfhkpis = get_last_df(path_kpis)
    dfhkpis = dfhkpis[
        (dfhkpis["ENTITY"].astype(str) == str(entity)) & (dfhkpis["PERIOD"].astype(str) == str(scenario))
    ].reset_index(drop=True)
    
    # Create style on var
    def create_style(value):
        if float(value) < 0:
            color = COLOR_RED
        elif float(value) == 0:
            color = COLOR_ORANGE
        elif float(value) > 0:
            color = COLOR_GREEN
        return {
            "color": color,
            "font-size": "20px",
            "font-weight": "700",
            "text-align": "center",
            "padding-top": "0px",
        }
    
    # Return card data
    def get_card_data(df, rubrique):
        value = "0 k€"
        var = "%" 
        style = 0
        
        # Filter
        tmp_df = df[df["RUBRIQUE_N1"] == rubrique].reset_index(drop=True)
        if len(tmp_df) > 0:
            value = tmp_df.loc[0, "VALUE_D"]
            var = tmp_df.loc[0, "VARP_D"]
            style = tmp_df.loc[0, "VARV"]
        return value, var, create_style(style)
        
    value1, var1, style1 = get_card_data(dfhkpis, "CHIFFRE_D'AFFAIRES")
    value2, var2, style2 = get_card_data(dfhkpis, "MARGE")
    value3, var3, style3 = get_card_data(dfhkpis, "EBE")
    value4, var4, style4 = get_card_data(dfhkpis, "BFR")
    value5, var5, style5 = get_card_data(dfhkpis, "CREANCES_CLIENTS")
    value6, var6, style6 = get_card_data(dfhkpis, "DETTES_FOURNISSEURS")
    

    return (
        value1,
        value2,
        value3,
        value4,
        value5,
        value6,
        var1,
        var2,
        var3,
        var4,
        var5,
        var6,
        style1,
        style2,
        style3,
        style4,
        style5,
        style6,
        fig1,
        fig2,
        fig3,
        fig4,
        ai1,
        ai2,
        ai3,
        ai4
    )

## Output

### Generate URL and show logs

In [ ]:
if __name__ == "__main__":
    app.run_server(proxy=f"http://127.0.0.1:{DASH_PORT}::https://app.naas.ai")